In [1]:
from os.path import join as pathjoin
from data_processing import *
from interpretation import *
from models import *
from training import *

Let's download all the data.

In [2]:
DATA_DIR = '/home/mlepekhin/data'
MODELS_DIR = '/home/mlepekhin/models'
MODEL_ID = 'allennlp_rubert_from_topic_generated'
CHECKPOINTS_DIR = pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints')
BEST_MODEL = pathjoin(CHECKPOINTS_DIR, 'best.th')

In [3]:
import pandas as pd

df1 = pd.read_csv(pathjoin(DATA_DIR, 'ru_train'))
df2 = pd.read_csv(pathjoin(DATA_DIR, 'min_gpt_bpe/ru_train_topic_big_prefixes.csv'))[['text', 'target']]
df1.append(df2).to_csv(pathjoin(DATA_DIR, "ru_adversarial_train"))

In [4]:
transformer_model = 'DeepPavlov/rubert-base-cased'
MAX_TOKENS = 512

In [5]:
train_dataset_reader = build_transformer_dataset_reader(transformer_model, lower=True)
val_dataset_reader = build_transformer_dataset_reader(transformer_model, lower=True)

train_data, dev_data = read_data(
    pathjoin(DATA_DIR, "ru_adversarial_train"), 
    pathjoin(DATA_DIR, "ru_test"),
    train_dataset_reader, 
    val_dataset_reader
)

vocab = build_vocab(train_data + dev_data)

train_data.index_with(vocab)
dev_data.index_with(vocab)

Reading data
<class 'data_processing.ClassificationDatasetReader'> /home/mlepekhin/data/ru_adversarial_train



Building the vocabulary


In [6]:
model = build_transformer_model(vocab, transformer_model)

Building the model


In [7]:
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
print(cuda_device)

0


In [8]:
!rm -rf {CHECKPOINTS_DIR}
!mkdir -p {CHECKPOINTS_DIR}

In [ ]:
train_loader, dev_loader = build_data_loaders(train_data, dev_data)

# You obviously won't want to create a temporary file for your training
# RU
#results, but for execution in binder for this course, we need to do this.

trainer = build_classifier_trainer(
    model,
    pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints'),
    train_loader,
    dev_loader,
    5,
    cuda_device=cuda_device
)
print("Starting training")
trainer.train()
print("Finished training")

You provided a validation dataset but patience was set to None, meaning that early stopping is disabled


Starting training


In [ ]:
!rm "{CHECKPOINTS_DIR}"/*.json "{CHECKPOINTS_DIR}"/model* "{CHECKPOINTS_DIR}"/training*
model.vocab.save_to_files(pathjoin(MODELS_DIR, MODEL_ID, 'vocab'))
!ls -lh "{CHECKPOINTS_DIR}"